In [75]:
import cv2 as cv2
import os
from skimage import io
import numpy as np
import matplotlib.pyplot as plt

In [76]:
def read_meteor(path, in_folder="data/meteor/", out_folder=None):
    mask = io.imread(in_folder + path + "_mask.png")

    img = io.imread(in_folder + path + ".tif")
    img[mask == 0] = 0
    img = ((img - img.min()) / (img.max() - img.min()) * 255).astype(np.uint8)

    if out_folder is not None:
        plt.imsave(out_folder + path + ".png", img, cmap='gray')

    return img, mask

In [77]:
def detect_meteor(img, path=None, threshold=50, dilation_factor=5, out_folder=None):
    img = (img > threshold).astype(np.uint8)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (dilation_factor, dilation_factor))
    dilated_img = cv2.dilate(img, kernel)

    edges = cv2.Canny(dilated_img*255, 1, 255)

    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=10)

    if out_folder is not None:
        plt.imsave(out_folder + path + "_dilated.png", dilated_img, cmap='gray')
        plt.imsave(out_folder + path + "_canny.png", edges, cmap='gray')

    if lines is None:
        if path is not None:
            print('Lines not found in "' + path + '".')
        else:
            print("Lines not found.")
        return None, dilated_img
    
    l = len(lines)
    lines = lines.squeeze()
    if l == 1:
        lines = np.array([lines])

    longest_line = max(lines, key=lambda line: (line[2] - line[0])**2 + (line[3] - line[1])**2)

    if out_folder is not None:
        x1, y1, x2, y2 = longest_line
        img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2.line(img_color, (x1, y1), (x2, y2), (0, 0, 255), 2)
        plt.imsave(out_folder + path + "_hough.png", img_color)

    return longest_line, dilated_img

In [78]:
def rotate_meteor(imgs, line, out_path=None):
    x1, y1, x2, y2 = line
    dx = x2 - x1
    dy = y2 - y1
    angle_radians = np.arctan2(dy, dx)
    angle_degrees = np.degrees(angle_radians)

    rotated_imgs = []
    for img in imgs:
        rows, cols = img.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle_degrees, 1)
        rotated_img = cv2.warpAffine(img, rotation_matrix, (cols, rows))
        rotated_imgs.append(rotated_img)
    
    if out_path is not None:
        for i in range(len(rotated_imgs)):
            rotated_img = rotated_imgs[i]
            plt.imsave(out_path + "_" + str(i) + ".png", rotated_img, cmap="gray")
    
    return rotated_imgs

In [79]:
def crop_meteor(img, mask, out_path=None):    
    x, y, w, h = cv2.boundingRect(mask)
    img = img[y:y+h, x:x+w]

    if out_path is not None:
        plt.imsave(out_path + ".png", img, cmap='gray')

    return img

In [80]:
def spectral_from_meteor(img, out_path=None):
    dft = cv2.dft(np.float32(img), flags=cv2.DFT_COMPLEX_OUTPUT)
    dft = np.fft.fftshift(dft)

    magnitude_spectral_full = cv2.magnitude(dft[:,:,0], dft[:,:,1])

    center_index = int(magnitude_spectral_full.shape[0] / 2)
    magnitude_1d = magnitude_spectral_full[center_index,:]
    
    if out_path is not None:
        plt.plot(magnitude_1d)
        plt.savefig(out_path + "_simple.png")
        plt.close()

    return magnitude_1d

In [81]:
def freq_dist_to_pixel_dist(freq_dist, magnitude_1d):
    pixel_dist = 1/ ( freq_dist / magnitude_1d.shape[0] )
    return pixel_dist

def pixel_dist_to_pixel_speed(pixel_dist, refresh_duration=100):
    meteor_speed_ms = pixel_dist / refresh_duration
    meteor_speed_s = meteor_speed_ms * 1000
    return meteor_speed_s

def pixel_speed_to_real_speed(meteor_speed_s, dist_per_pixel=1000):
    real_speed = meteor_speed_s * dist_per_pixel
    return real_speed

def meteor_speed(magnitude_1d, path=None, out_folder=None):
    magnitude_1d_begin = np.insert(magnitude_1d, 0, magnitude_1d[0]) # 0 begin
    magnitude_1d_end   = np.append(magnitude_1d, magnitude_1d[-1]) # 0 end

    derivate = magnitude_1d_end - magnitude_1d_begin
    supremums = np.all([derivate[:-1] > 0, derivate[1:] < 0], axis=0)

    x_supremums = np.where(supremums)[0]
    y_supremums = np.array(magnitude_1d[supremums])

    indexes = np.flip(np.argsort(y_supremums))

    if out_folder is not None:
        plt.plot(magnitude_1d)
        plt.plot(x_supremums[indexes[:3]], y_supremums[indexes[:3]], 'r*')
        plt.savefig(out_folder + path + "_supremums.png")
        plt.close()

    if len(x_supremums) < 2:
        if path is not None:
            print('2 Supremums not found in "' + path + '".')
        else:
            print("2 Supremums not found.")
        return None

    center_index = x_supremums[indexes[0]]
    max_index = x_supremums[indexes[1]]
    dist_freq = abs(center_index - max_index)

    dist_pixel = freq_dist_to_pixel_dist(dist_freq, magnitude_1d)
    pixel_speed = pixel_dist_to_pixel_speed(dist_pixel)
    real_speed = pixel_speed_to_real_speed(pixel_speed)

    return real_speed

In [82]:
folder_path = 'data/meteor'

paths = []
for filename in os.listdir(folder_path):
    if filename.endswith('.tif'):
        file_path = os.path.join(folder_path, filename)
        paths.append(file_path.split('.')[0].split('/')[-1])

print(len(paths))

867


In [83]:
# GLOBAL FRAMEWORK
errors = 0

f = open("data/speeds.csv", 'w')
f.write("name;speed;error\n")

for path in paths:
    f.write(path + ';')

    img, mask = read_meteor(path, out_folder="data/cleaned/")
    line, dilated = detect_meteor(img, path, out_folder="data/lines/")
    if line is not None:
        img, mask = rotate_meteor([img, mask], line, out_path="data/rotations/" + path)
        img = crop_meteor(img, mask, out_path="data/cropped/" + path)
        magnitude = spectral_from_meteor(img, out_path="data/spectral/" + path)
        speed = meteor_speed(magnitude, path, out_folder="data/spectral/")
        if speed is not None:
            f.write(str(speed) + ";\n")
        else:
            errors += 1
            f.write("NaN;supremums not found\n")

    else:
        errors += 1
        f.write("NaN;lines not found\n")

f.close()

print(errors, "images cannot give a result.")

Lines not found in "20160813_222915_730_723_816_409_690".
Lines not found in "20121112_195650_609_249_350_590_803".
Lines not found in "20201124_203911_733_577_699_546_627".
Lines not found in "20161002_221520_186_100_187_651_942".
Lines not found in "20160812_230311_962_179_251_18_238".


Lines not found in "20160812_220733_103_661_781_678_980".
Lines not found in "20181206_195621_026_756_850_737_980".
Lines not found in "20161213_034228_045_320_578_401_530".
Lines not found in "20160812_223238_100_596_693_382_622".
Lines not found in "20200214_055352_328_787_907_625_874".
Lines not found in "20190508_020326_525_350_489_777_880".
Lines not found in "20160812_003155_114_120_225_1062_1214".
Lines not found in "20150507_021540_492_611_668_365_835".
Lines not found in "20150919_215226_985_275_736_560_683".
Lines not found in "20180620_022603_843_316_433_219_333".
Lines not found in "20180916_214300_394_244_563_196_368".
Lines not found in "20210311_041043_875_709_840_1030_1217".
Lines not found in "20180424_021120_588_434_681_56_834".
Lines not found in "20150811_015128_648_69_173_65_227".
Lines not found in "20201121_041031_692_577_780_627_803".
Lines not found in "20160808_215227_254_51_207_460_512".
Lines not found in "20170320_021331_182_196_323_699_914".
Lines not foun